In [2]:
from art.utils.benchmarking.generate_comparison_table import generate_comparison_table
from art.utils.benchmarking.types import BenchmarkedModelKey

table = generate_comparison_table(
    project="2048-multi-turn",
    benchmark_keys=[
        BenchmarkedModelKey("007", "train"),
        BenchmarkedModelKey("008", "train", [53]),
        BenchmarkedModelKey("009", "train", [0, -2, -1])
    ],
    metrics=["reward", "max_value", "board_value"]
)

print(table.to_markdown())

|    |   Model | Split   |   Step |   reward |   max_value |   board_value |
|---:|--------:|:--------|-------:|---------:|------------:|--------------:|
|  0 |     007 | train   |   0000 | 0.500402 |          32 |            58 |
|  1 |     007 | train   |   0001 | 2.00032  |          32 |            54 |
|  2 |     007 | train   |   0002 | 2.00038  |          32 |            64 |
|  3 |     007 | train   |   0003 | 0.500034 |          32 |            68 |
|  4 |     007 | train   |   0004 | 0.500139 |          32 |            54 |
|  5 |     007 | train   |   0005 | 0.500105 |          32 |            68 |
|  6 |     007 | train   |   0006 | 2.00028  |          32 |            58 |
|  7 |     008 | train   |   0053 | 2.0007   |          32 |            53 |
|  8 |     009 | train   |   0000 | 0.500467 |         128 |           202 |
|  9 |     009 | train   |   0376 | 0.967989 |          64 |           213 |
| 10 |     009 | train   |   0377 | 1.48512  |          96 |           230 |